In [ ]:
import pandas as pd
import numpy as np

# Carga de datos
df_viajes_aereos = pd.read_csv("./data/dataset_viajes.csv", index_col="Id_vuelo")

# Información general
df_viajes_aereos.info()

# Descripción de variables numéricas
df_viajes_aereos.describe()

# Columnas del DataFrame
df_viajes_aereos.columns

# Distribución de 3 columnas escogidas (ejemplo: Distancia, Consumo_keroseno, Precio)
df_viajes_aereos['Distancia'].value_counts(dropna=False).head(10)
df_viajes_aereos['Consumo_keroseno'].value_counts(dropna=False).head(10)
df_viajes_aereos['Precio'].value_counts(dropna=False).head(10)
# Número de duplicados
duplicados = df_viajes_aereos.duplicated().sum()

# Filas duplicadas por compañía
df_viajes_aereos['Compañía'].value_counts()  # luego compara con duplicados
df_viajes_aereos = df_viajes_aereos.drop_duplicates(keep='last')
# Columnas con nulos
df_viajes_aereos.isnull().sum()

# Porcentaje de nulos por columna
df_viajes_aereos.isnull().mean() * 100
df_viajes_aereos['Compañía'].unique()
df_viajes_aereos = df_viajes_aereos.dropna(thresh=df_viajes_aereos.shape[1]-2)
df_viajes_aereos['Consumo_keroseno'].fillna(df_viajes_aereos['Consumo_keroseno'].mean(), inplace=True)
df_viajes_aereos.drop(columns=['Columna_no_reconstruible'], inplace=True)
df_viajes_aereos.drop(columns=['Columna_no_reconstruible'], inplace=True)
# Ejemplo: eliminar espacios en Compañía
df_viajes_aereos['Compañía'] = df_viajes_aereos['Compañía'].str.strip()
def clasificar_distancia(km):
    if km > 9000:
        return 'LD'
    elif 1500 <= km <= 9000:
        return 'MD'
    else:
        return 'R'

df_viajes_aereos['Categoria'] = df_viajes_aereos['Distancia'].apply(clasificar_distancia)
df_viajes_aereos['IC'] = df_viajes_aereos['Consumo_keroseno'] / df_viajes_aereos['Distancia']
# IC global por compañía
ic_por_empresa = df_viajes_aereos.groupby('Compañía')['IC'].sum()

# 3 más contaminantes
top3_contaminantes = ic_por_empresa.nlargest(3)
df_viajes_aereos.groupby(['Categoria','Compañía'])['IC'].sum().groupby(level=0, group_keys=False).nlargest(1)
df_viajes_aereos.groupby(['Categoria','Compañía'])['IC'].sum().groupby(level=0, group_keys=False).nlargest(1)
ic_por_destino = df_viajes_aereos.groupby('Destino')['IC'].sum() / df_viajes_aereos.groupby('Destino')['IC'].count()
destinos_top = ic_por_destino.nlargest(2)
ic_por_destino = df_viajes_aereos.groupby('Destino')['IC'].sum() / df_viajes_aereos.groupby('Destino')['IC'].count()
destinos_top = ic_por_destino.nlargest(2)
for destino in destinos_top.index:
    print(df_viajes_aereos[df_viajes_aereos['Destino']==destino]['Compañía'].value_counts().head(1))
df_viajes_extra = pd.read_csv("./data/dataset_viajes_extra.csv", index_col="Id_vuelo")
df_viajes = pd.concat([df_viajes_aereos, df_viajes_extra])
df_viajes.info()
df_viajes['IC'] = df_viajes['IC'].fillna(df_viajes['Consumo_keroseno']/df_viajes['Distancia'])
df_ingresos = pd.read_csv("./data/dataset_ingresos.csv")
df_final = df_viajes.merge(df_ingresos, left_index=True, right_on="Id_vuelo", how="left")
df_final.set_index("Id_vuelo", inplace=True)
df_final.groupby('Compañía')['Ingresos'].sum()
df_final.groupby(['Compañía','Origen'])['Ingresos'].sum()
df_final.groupby(['Compañía','Destino'])['Ingresos'].sum()
empresas_top = ic_por_empresa.nlargest(2).index
impuesto = df_final[df_final['Compañía'].isin(empresas_top)]['Ingresos'].sum() * 0.002
df_final['Beneficio_por_km'] = df_final['Ingresos'] / df_final['Distancia']
df_final.groupby(['Compañía','Origen','Destino']).agg(
    ingresos_total=('Ingresos','sum'),
    consumo_medio=('Consumo_keroseno','mean'),
    IC_medio=('IC','mean'),
    numero_viajes=('Distancia','count')
)
